# MATH 406 Assignment 1

**Name:** James Wu\
**Student number:** 92277235

In [1]:
### IMPORTS ###
import numpy as np

## Q1

### Q1(a)

With 2 points, we use linear interpolation. Solving for $a$ and $b$ in the interpolating function $L(x) = ax + b$ using $L(0) = 0$ and $L(1) = 1$ gives us
$$L(x) = x$$
on $[0,1]$. Then $f(1/2) \approx L(1/2)$, so
$$\boxed{f(1/2) \approx 1/2}$$

In [5]:
# Given data
x = np.array([0, 1])
y = np.array([0, 1])

# Polynomial coeffs
c = np.linalg.solve(np.vander(x), y)
print(f'Polynomial coeffs: {c}')

# Evaluate f(1/2)
ans = np.polyval(c, 1/2)
print(f'f(1/2): {ans}')

Polynomial coeffs: [1. 0.]
f(1/2): 0.5


### Q1(b)

With 3 points, we use quadratic interpolation. Solving for $a$, $b$, and $c$ in the interpolating function $Q(x) = ax^2 + bx + c$ using $Q(0) = 0$, $Q(1) = 1$, and $Q(-1) = 1/2$ gives us
$$Q(x) = \frac{3}{4}x^2 + \frac{1}{4}x$$
on $[-1,1]$. Then $f(1/2) \approx Q(1/2)$, so
$$\boxed{f(1/2) \approx 5/16}$$

In [7]:
# Given data
x = np.array([0, 1, -1])
y = np.array([0, 1, 1/2])

# Polynomial coeffs
c = np.linalg.solve(np.vander(x), y)
print(f'Polynomial coeffs: {c}')

# Evaluate f(1/2)
ans = np.polyval(c, 1/2)
print(f'f(1/2): {ans}')

Polynomial coeffs: [0.75 0.25 0.  ]
f(1/2): 0.3125


### Q1(c)

With 4 points, we use cubic interpolation. Solving for $a$, $b$, $c$, and $d$ in the interpolating function $C(x) = ax^3 + bx^2 + cx + d$ using $C(0) = 0$, $C(1) = 1$, $C(-1) = 1/2$, and $C(2) = 2$ gives us
$$C(x) = -\frac{1}{4}x^3 + \frac{3}{4}x^2 + \frac{1}{2}x$$
$$C'(x) = -\frac{3}{4}x^2 + \frac{3}{2}x + \frac{1}{2}$$
on $[-1,2]$. Then $f'(0) \approx C'(0)$, so
$$\boxed{f'(0) \approx 1/2}$$

In [9]:
# Given data
x = np.array([0, 1, -1, 2])
y = np.array([0, 1, 1/2, 2])

# Polynomial coeffs
c = np.linalg.solve(np.vander(x), y)
print(f'Polynomial coeffs: {c}')

# Differentiate polynomial
dc = np.polyder(c)
print(f'Differentiated coeffs: {dc}')

# Evaluate f'(0)
ans = np.polyval(dc, 0)
print(f"f'(0): {ans}")

Polynomial coeffs: [-0.25  0.75  0.5   0.  ]
Differentiated coeffs: [-0.75  1.5   0.5 ]
f'(0): 0.5


### Q1(d)

With 3 points, we use quadratic interpolation. Solving for $a$, $b$, and $c$ in the interpolating function $Q(x) = ax^2 + bx + c$ using $Q(0) = 1/2$, $Q(1) = 1$, and $Q(-1) = -1$ gives us
$$Q(x) = -\frac{1}{2}x^2 + x + \frac{1}{2}$$
on $[-1,1]$. Using the quadratic equation for $Q(x) = 0$ gets us the roots
$$x = \frac{-1 \pm \sqrt{1 + 1}}{-1} = 1 \pm \sqrt{2}$$
Although $1 - \sqrt{2} \in [-1,1]$, we have $1 + \sqrt{2} \notin [-1,1]$. Since our interpolation is only valid on $[-1,1]$, we approximate a root as
$$\boxed{x \approx 1 - \sqrt{2} \approx -0.414}$$

In [11]:
# Given data
x = np.array([0, 1, -1])
y = np.array([1/2, 1, -1])

# Polynomial coeffs
c = np.linalg.solve(np.vander(x), y)
print(f'Polynomial coeffs: {c}')

# Roots of quadratic
ans = np.roots(c)
print(f'Roots: {ans}')

Polynomial coeffs: [-0.5  1.   0.5]
Roots: [ 2.41421356 -0.41421356]
